# Data preprocessing

This notebook is used to clean data, separate into two dedicated datasets based on education level, and perform some low level statistical inpection.

**Load data**

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./raw_data.csv')

In [2]:
df.columns

Index(['YEAR', 'LocationAbbr', 'LocationDesc', 'TopicType', 'TopicDesc',
       'MeasureDesc', 'DataSource', 'Response', 'Data_Value_Unit',
       'Data_Value_Type', 'Data_Value', 'Data_Value_Footnote_Symbol',
       'Data_Value_Footnote', 'Data_Value_Std_Err', 'Low_Confidence_Limit',
       'High_Confidence_Limit', 'Sample_Size', 'Gender', 'Race', 'Age',
       'Education', 'GeoLocation', 'TopicTypeId', 'TopicId', 'MeasureId',
       'StratificationID1', 'StratificationID2', 'StratificationID3',
       'StratificationID4', 'SubMeasureID', 'DisplayOrder'],
      dtype='object')

**Keep columns of interest**

In [3]:
df = df[['YEAR', 'LocationAbbr', 'LocationDesc', 'TopicDesc', 'MeasureDesc',
        'Response', 'Data_Value', 'Data_Value_Std_Err', 'Low_Confidence_Limit',
        'High_Confidence_Limit', 'Sample_Size', 'Gender', 'Education']]

In [4]:
df.head(10)

,YEAR,LocationAbbr,LocationDesc,TopicDesc,MeasureDesc,Response,Data_Value,Data_Value_Std_Err,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Gender,Education
0,2017,AZ,Arizona,Cessation (Youth),Percent of Current Smokers Who Want to Quit,NaN,NaN,NaN,NaN,NaN,NaN,Overall,Middle School
1,2017,AZ,Arizona,Cessation (Youth),Percent of Current Smokers Who Want to Quit,NaN,NaN,NaN,NaN,NaN,NaN,Male,Middle School
2,2017,AZ,Arizona,Cessation (Youth),Percent of Current Smokers Who Want to Quit,NaN,NaN,NaN,NaN,NaN,NaN,Female,Middle School
3,2017,AZ,Arizona,Cessation (Youth),Quit Attempt in Past Year Among Current Cigare...,NaN,NaN,NaN,NaN,NaN,NaN,Overall,Middle School
4,2017,AZ,Arizona,Cessation (Youth),Quit Attempt in Past Year Among Current Cigare...,NaN,NaN,NaN,NaN,NaN,NaN,Male,Middle School
5,2017,AZ,Arizona,Cessation (Youth),Quit Attempt in Past Year Among Current Cigare...,NaN,NaN,NaN,NaN,NaN,NaN,Female,Middle School
6,2017,AZ,Arizona,Cigarette Use (Youth),Smoking Status,Current,2.4,0.5,1.4,3.4,1346.0,Overall,Middle School
7,2017,AZ,Arizona,Cigarette Use (Youth),Smoking Status,Ever,12.7,2.0,8.7,16.7,1248.0,Overall,Middle School
8,2017,AZ,Arizona,Cigarette Use (Youth),Smoking Status,Frequent,0.3,0.2,0.0,0.7,1346.0,Overall,Middle School
9,2017,AZ,Arizona,Cigarette Use (Youth),Smoking Status,Current,2.9,0.7,1.5,4.3,679.0,Male,Middle School


**Remove the "(Youth)" text in TopicDesc column**

In [5]:
import re

df['TopicDesc'] = [re.sub(' \(Youth\)', '', x) for x in df.TopicDesc]

In [6]:
df.TopicDesc.head()

0    Cessation
1    Cessation
2    Cessation
3    Cessation
4    Cessation
Name: TopicDesc, dtype: object

**Simplify MeasureDesc values (less text)**

In [7]:
df.MeasureDesc.unique().tolist()

['Percent of Current Smokers Who Want to Quit',
 'Quit Attempt in Past Year Among Current Cigarette Smokers',
 'Smoking Status',
 'User Status']

In [8]:
df['MeasureDesc'] = ['Want to Quit' if x == 'Percent of Current Smokers Who Want to Quit' else 
                     'Quit Attempt' if x == 'Quit Attempt in Past Year Among Current Cigarette Smokers' else x
                    for x in df.MeasureDesc]

In [9]:
df.MeasureDesc.head()

0    Want to Quit
1    Want to Quit
2    Want to Quit
3    Quit Attempt
4    Quit Attempt
Name: MeasureDesc, dtype: object

**Inspect observations by state**

In [21]:
df.groupby('LocationDesc').count().sort_values(by='YEAR', ascending=False)[['YEAR']]

,YEAR
LocationDesc,
Mississippi,651
Alabama,426
New Jersey,411
Wisconsin,408
Connecticut,405
North Carolina,402
West Virginia,378
North Dakota,378
Delaware,354


There are a handful of states (16) that have over 300 observations in the data. Because the data for the remaining states is so sparse, it will be better to focus on the more better represented states.

**Split data into Middle School and High School sets**

In [10]:
ms_data = df[df.Education == 'Middle School']
hs_data = df[df.Education == 'High School']

**Inspect sample size statistics**

In [11]:
ms_data.groupby(['LocationAbbr', 'TopicDesc', 'Gender']).agg([min,max,np.mean,np.std])[['Sample_Size']]

Sample_Size                       \
                                                   min     max         mean   
LocationAbbr TopicDesc             Gender                                     
AL           Cessation             Female         50.0   106.0    73.300000   
                                   Male           50.0   137.0    87.400000   
                                   Overall        50.0   243.0   122.705882   
             Cigarette Use         Female        499.0  1479.0   784.888889   
                                   Male          469.0  1465.0   743.111111   
                                   Overall       972.0  2953.0  1532.518519   
             Smokeless Tobacco Use Female        535.0  1494.0   809.444444   
                                   Male          503.0  1483.0   772.111111   
                                   Overall      1042.0  2986.0  1586.296296   
AR           Cessation             Female         67.0    98.0    79.428571   
                                   Male           59.0   104.0    85.166667   
                                   Overall       125.0   203.0   160.428571   
             Cigarette Use         Female        208.0  1201.0   914.000000   
                                   Male          192.0  1165.0   899.769231   
                                   Overall       400.0  2387.0  1824.384615   
             Smokeless Tobacco Use Female        211.0  1205.0   835.933333   
                                   Male          194.0  1174.0   822.466667   
                                   Overall       406.0  2405.0  1668.800000   
AZ           Cessation             Female         64.0    82.0    70.000000   
                                   Male           50.0   103.0    69.000000   
                                   Overall        96.0   186.0   135.636364   
             Cigarette Use         Female        620.0  1944.0  1029.148148   
                                   Male          620.0  1997.0  1024.592593   
                                   Overall      1248.0  3979.0  2065.185185   
             Smokeless Tobacco Use Female        662.0  1965.0  1049.666667   
                                   Male          666.0  2009.0  1056.666667   
                                   Overall      1335.0  4013.0  2118.625000   
CA           Cessation             Female         75.0   203.0   136.250000   
                                   Male           79.0   188.0   130.500000   
                                   Overall       154.0   394.0   268.500000   
...                                                ...     ...          ...   
VT           Smokeless Tobacco Use Female        727.0  2326.0  1302.666667   
                                   Male          733.0  2312.0  1300.666667   
                                   Overall      1462.0  4662.0  2613.111111   
WI           Cessation             Female         55.0    83.0    67.000000   
                                   Male           51.0    73.0    63.500000   
                                   Overall        50.0   156.0    93.285714   
             Cigarette Use         Female        595.0   936.0   795.259259   
                                   Male          627.0   948.0   810.518519   
                                   Overall      1227.0  1864.0  1610.703704   
             Smokeless Tobacco Use Female        611.0   955.0   810.111111   
                                   Male          651.0   962.0   830.074074   
                                   Overall      1269.0  1885.0  1645.333333   
WV           Cessation             Female         65.0   726.0   187.833333   
                                   Male           64.0   628.0   164.916667   
                                   Overall        66.0  1361.0   297.266667   
             Cigarette Use         Female        733.0  4910.0  1412.541667   
                                   Male          713.0  4724.0  1360.750000   
              

In [12]:
ms_data.groupby(['LocationAbbr', 'TopicDesc', 'Gender']).agg([min,max])[['Sample_Size']].max()

Sample_Size  min    21830.0
             max    26236.0
dtype: float64

Maximum Middle School sample size is 26,236

In [13]:
ms_data.groupby(['LocationAbbr', 'TopicDesc', 'Gender']).agg([min,max])[['Sample_Size']].min()

Sample_Size  min    50.0
             max    50.0
dtype: float64

Minimum Middle School sample size is 50

In [14]:
hs_data.groupby(['LocationAbbr', 'TopicDesc', 'Gender']).agg([min,max,np.mean,np.std])[['Sample_Size']]

Sample_Size                       \
                                                   min     max         mean   
LocationAbbr TopicDesc             Gender                                     
AL           Cessation             Female         68.0   199.0   113.411765   
                                   Male           92.0   192.0   141.647059   
                                   Overall       160.0   390.0   256.117647   
             Cigarette Use         Female        613.0  1256.0   765.814815   
                                   Male          602.0  1174.0   735.222222   
                                   Overall      1257.0  2431.0  1505.148148   
             Smokeless Tobacco Use Female        619.0  1271.0   777.629630   
                                   Male          610.0  1197.0   751.333333   
                                   Overall      1279.0  2469.0  1533.037037   
AR           Cessation             Female        132.0   219.0   177.714286   
                                   Male          162.0   241.0   201.714286   
                                   Overall       294.0   446.0   381.000000   
             Cigarette Use         Female        433.0  1108.0   903.846154   
                                   Male          494.0  1118.0   881.153846   
                                   Overall       929.0  2252.0  1794.692308   
             Smokeless Tobacco Use Female        434.0  1103.0   849.000000   
                                   Male          481.0  1110.0   835.000000   
                                   Overall       917.0  2242.0  1694.266667   
AZ           Cessation             Female         98.0   150.0   117.200000   
                                   Male          104.0   151.0   119.600000   
                                   Overall       203.0   301.0   237.200000   
             Cigarette Use         Female        787.0   975.0   895.333333   
                                   Male          738.0   923.0   824.000000   
                                   Overall      1527.0  1899.0  1723.888889   
             Smokeless Tobacco Use Female        812.0   931.0   868.166667   
                                   Male          753.0   820.0   788.500000   
                                   Overall      1567.0  1758.0  1660.500000   
CA           Cessation             Female        224.0   825.0   501.000000   
                                   Male          286.0   900.0   570.250000   
                                   Overall       510.0  1731.0  1074.250000   
...                                                ...     ...          ...   
VI           Smokeless Tobacco Use Female        579.0   645.0   610.833333   
                                   Male          446.0   516.0   480.666667   
                                   Overall      1027.0  1166.0  1095.000000   
VT           Cessation             Female        129.0   137.0   133.000000   
                                   Male          117.0   127.0   122.000000   
                                   Overall       246.0   265.0   255.500000   
             Cigarette Use         Female        679.0   693.0   688.333333   
                                   Male          668.0   695.0   686.000000   
                                   Overall      1351.0  1396.0  1381.000000   
             Smokeless Tobacco Use Female        698.0   705.0   702.666667   
                                   Male          702.0   710.0   707.333333   
                                   Overall      1405.0  1423.0  1417.000000   
WI           Cessation             Female         61.0   228.0   141.214286   
                                   Male           87.0   182.0   136.285714   
                                   Overall        71.0   411.0   253.000000   
             Cigarette Use         Female        549.0   922.0   752.208333   
                                   Male          595.0   945.0   736.291667   
              

In [15]:
hs_data.groupby(['LocationAbbr', 'TopicDesc', 'Gender']).agg([min,max])[['Sample_Size']].max()

Sample_Size  min    32136.0
             max    36910.0
dtype: float64

Maximum High School sample size is 36,910

In [16]:
hs_data.groupby(['LocationAbbr', 'TopicDesc', 'Gender']).agg([min,max])[['Sample_Size']].min()

Sample_Size  min    50.0
             max    60.0
dtype: float64

Minimum High School sample size is 50. It appears that 50 is the cutoff sample size for the data to be included.

The sample sizes vary quite a bit, so the confidence intervals will be useful to show how meaningful the smoking statistics are.

**Write High School and Middle School dataframes to separate csv files**

In [17]:
# ms_data.to_csv('./ms_data.csv', index=False)
# hs_data.to_csv('./hs_data.csv', index=False)

**Perform further analysis in R**